In [3]:
import sys

In [7]:
a = [1, 2, 3]

In [8]:
b = a

In [9]:
sys.getrefcount(a)

3

In [4]:
from multiprocessing import Process, Manager, Pipe, cpu_count

In [2]:
def func(d, i):
    d.append(i)

In [3]:
manager = Manager()
d = manager.list()
process_list = [Process(target=func, args=(d, i)) for i in range(7)]

In [4]:
for proc in process_list:
    proc.start()

In [5]:
for proc in process_list:
    proc.join()

In [6]:
process_list[0].pid

4281

In [7]:
list(d)

[0, 1, 2, 3, 4, 5, 6]

In [8]:
manager.shutdown()

In [12]:
import os, time


In [13]:
def proc(idx, n, pipe):
    val = 0
    if idx == 0:
        pipe.send(0)
    while val < n:
        val = pipe.recv()
        print(val, os.getpid())
        time.sleep(1)
        val += 1
        
        pipe.send(val)

In [14]:
pipe_l, pipe_r = Pipe()

In [15]:
proc1 = Process(target=proc, args=(0, 10, pipe_l))
proc2 = Process(target=proc, args=(1, 10, pipe_r))

In [16]:
proc1.start()
proc2.start()

0 4347
1 4346
2 4347
3 4346
4 4347
5 4346
6 4347
7 4346
8 4347
9 4346
10 4347


In [17]:
proc1.join()
proc2.join()

In [18]:
proc1.pid

4346

In [1]:
%env OMP_NUM_THREADS=1

import numpy as np

env: OMP_NUM_THREADS=1


In [10]:
n = 4_000
A = np.random.randn(n, n)
B = np.random.randn(n, n)

In [11]:
%%time

C = A @ B

CPU times: user 3.83 s, sys: 31.9 ms, total: 3.86 s
Wall time: 3.86 s


In [12]:
num_proc = cpu_count()

In [18]:
block_size = n // num_proc
num_blocks = n // block_size
block_size += bool(n % num_proc)
i_pos = [min(i * block_size, n) for i in range(num_blocks + 1)]

In [19]:
i_pos

[0, 667, 1334, 2001, 2668, 3335, 4000]

In [21]:
def matmul(A, B, d, i):
    d[i] = A @ B

In [31]:
manager = Manager()
d = manager.dict()

proc_list = [Process(target=matmul,
                     args=(A[i_pos[i]:i_pos[i + 1]], B, d, i)) for i in range(num_proc)]

In [32]:
%%time

for proc in proc_list:
    proc.start()
    
for proc in proc_list:
    proc.join()
    
res = [d[i] for i in range(len(d))]
C_p = np.vstack(res)

CPU times: user 114 ms, sys: 196 ms, total: 311 ms
Wall time: 1.4 s


In [48]:
manager.shutdown()

In [33]:
from multiprocessing import Array
import ctypes

In [42]:
C_shared = Array(ctypes.c_double, n * n, lock=False)

In [43]:
C = np.frombuffer(C_shared, dtype=np.float64).reshape(n, n)

In [44]:
def matmul_shared(C_shared, n, i1, i2):
    C = np.frombuffer(C_shared, dtype=np.float64).reshape(n, n)
    C[i1:i2] = A[i1:i2] @ B

In [45]:
proc_list = [Process(target=matmul_shared,
                     args=(C_shared, n, i_pos[i], i_pos[i+1])) for i in range(num_proc)]

In [46]:
%%time

for proc in proc_list:
    proc.start()
    
for proc in proc_list:
    proc.join()

CPU times: user 3.68 ms, sys: 72.6 ms, total: 76.3 ms
Wall time: 874 ms


In [47]:
3.86 / 0.874

4.416475972540046

In [50]:
del C_shared

In [57]:
def gcd(x, y):
    while (y):
        x, y = y, x % y
    return x

def phi(n):
    cnt = 0
    for i in range(1, n):
        if gcd(i, n) == 1:
            cnt += 1
    return cnt

In [60]:
a = np.random.randint(low=1, high=10**4, size=1000)

In [62]:
%%time

[phi(n) for n in a]

CPU times: user 6.95 s, sys: 0 ns, total: 6.95 s
Wall time: 6.94 s


[7224,
 3304,
 2352,
 1944,
 4200,
 288,
 6142,
 438,
 3960,
 5240,
 166,
 3922,
 3392,
 2016,
 4892,
 5760,
 36,
 6960,
 1152,
 2560,
 2604,
 306,
 5568,
 3676,
 720,
 328,
 2520,
 2856,
 8820,
 4240,
 4000,
 100,
 5184,
 3320,
 3040,
 816,
 1236,
 618,
 7176,
 504,
 270,
 3672,
 1846,
 7296,
 1068,
 3288,
 3960,
 320,
 4760,
 3558,
 380,
 4270,
 544,
 2656,
 396,
 2688,
 8856,
 1912,
 8184,
 9718,
 7040,
 1000,
 1332,
 2208,
 6460,
 3824,
 8092,
 3690,
 552,
 660,
 1020,
 320,
 1440,
 528,
 6424,
 862,
 3600,
 2448,
 6240,
 268,
 960,
 4284,
 264,
 8232,
 2016,
 4284,
 1536,
 1080,
 4230,
 768,
 3000,
 2292,
 336,
 3120,
 1952,
 5800,
 5232,
 1320,
 3528,
 4032,
 2940,
 3520,
 1104,
 2984,
 2552,
 5260,
 3984,
 9036,
 6720,
 6720,
 1980,
 2016,
 1400,
 5502,
 5728,
 2688,
 316,
 1808,
 5400,
 2096,
 1272,
 1584,
 424,
 108,
 2880,
 4272,
 1872,
 120,
 1872,
 1872,
 5280,
 720,
 2528,
 5916,
 7884,
 3840,
 1440,
 2496,
 3040,
 4404,
 1920,
 3888,
 1040,
 192,
 1080,
 1920,
 1728,
 520

In [63]:
from multiprocessing import Pool

In [66]:
%%time

with Pool(processes=cpu_count()) as pool:
    res = pool.map(phi, a)

CPU times: user 71.8 ms, sys: 80.5 ms, total: 152 ms
Wall time: 1.4 s


In [65]:
res

[7224,
 3304,
 2352,
 1944,
 4200,
 288,
 6142,
 438,
 3960,
 5240,
 166,
 3922,
 3392,
 2016,
 4892,
 5760,
 36,
 6960,
 1152,
 2560,
 2604,
 306,
 5568,
 3676,
 720,
 328,
 2520,
 2856,
 8820,
 4240,
 4000,
 100,
 5184,
 3320,
 3040,
 816,
 1236,
 618,
 7176,
 504,
 270,
 3672,
 1846,
 7296,
 1068,
 3288,
 3960,
 320,
 4760,
 3558,
 380,
 4270,
 544,
 2656,
 396,
 2688,
 8856,
 1912,
 8184,
 9718,
 7040,
 1000,
 1332,
 2208,
 6460,
 3824,
 8092,
 3690,
 552,
 660,
 1020,
 320,
 1440,
 528,
 6424,
 862,
 3600,
 2448,
 6240,
 268,
 960,
 4284,
 264,
 8232,
 2016,
 4284,
 1536,
 1080,
 4230,
 768,
 3000,
 2292,
 336,
 3120,
 1952,
 5800,
 5232,
 1320,
 3528,
 4032,
 2940,
 3520,
 1104,
 2984,
 2552,
 5260,
 3984,
 9036,
 6720,
 6720,
 1980,
 2016,
 1400,
 5502,
 5728,
 2688,
 316,
 1808,
 5400,
 2096,
 1272,
 1584,
 424,
 108,
 2880,
 4272,
 1872,
 120,
 1872,
 1872,
 5280,
 720,
 2528,
 5916,
 7884,
 3840,
 1440,
 2496,
 3040,
 4404,
 1920,
 3888,
 1040,
 192,
 1080,
 1920,
 1728,
 520